In [1]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
dataset = pd.DataFrame()

In [13]:
data_path = 'Data_Train'

In [14]:
people = []
for person in os.listdir(data_path):
    if '.txt' in person: 
        people.append(person)
for window_length in range(1,5):
    for person in people: 
        name = person.split('R')[0]
        print (name)
        file = data_path+"/"+person
        temp = pd.read_csv(file,delimiter= " ", header = None)
        print (person,' is read',end = '\t')
        if 2 in temp[max(temp.columns)].unique():
            print ('Adding {} to dataset'.format(person),end = '\t')
            temp.columns = ['time','A_F','A_V','A_L','L_F','L_V','L_L','T_F','T_V','T_L','Action']    
#             temp = label_prefog(temp,window_length).reset_index(drop=True)
            temp['name'] = name
            print ('{} is labelled'.format(person))
            dataset = pd.concat([dataset,temp],axis = 0)

        print ('')
    dataset.reset_index(drop =True,inplace=True) 
    to_path = data_path + "\\raw_labelled"
    to_name = to_path +"\\win_"+str(window_length)+".csv"
     
    dataset.to_csv(to_name,index = False)


display(dataset.head())

S10
S10R01.txt  is read	
S02
S02R01.txt  is read	Adding S02R01.txt to dataset	S02R01.txt is labelled

S02
S02R02.txt  is read	Adding S02R02.txt to dataset	S02R02.txt is labelled

S06
S06R01.txt  is read	Adding S06R01.txt to dataset	S06R01.txt is labelled

S06
S06R02.txt  is read	
S04
S04R01.txt  is read	
S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled

S01
S01R02.txt  is read	Adding S01R02.txt to dataset	S01R02.txt is labelled

S03
S03R02.txt  is read	Adding S03R02.txt to dataset	S03R02.txt is labelled

S03
S03R03.txt  is read	
S03
S03R01.txt  is read	Adding S03R01.txt to dataset	S03R01.txt is labelled

S07
S07R02.txt  is read	Adding S07R02.txt to dataset	S07R02.txt is labelled

S07
S07R01.txt  is read	Adding S07R01.txt to dataset	S07R01.txt is labelled

S09
S09R01.txt  is read	Adding S09R01.txt to dataset	S09R01.txt is labelled

S05
S05R01.txt  is read	Adding S05R01.txt to dataset	S05R01.txt is labelled

S05
S05R02.txt  is read	Adding S05R02.txt to dataset	

,time,A_F,A_V,A_L,L_F,L_V,L_L,T_F,T_V,T_L,Action,name
0,15,50,39,-970,0,0,0,0,0,0,0,S02
1,31,50,39,-960,0,0,0,0,0,0,0,S02
2,46,50,29,-960,0,0,0,0,0,0,0,S02
3,62,50,29,-960,0,0,0,0,0,0,0,S02
4,78,40,29,-960,0,0,0,0,0,0,0,S02


In [15]:
dataset

,time,A_F,A_V,A_L,L_F,L_V,L_L,T_F,T_V,T_L,Action,name
0,15,50,39,-970,0,0,0,0,0,0,0,S02
1,31,50,39,-960,0,0,0,0,0,0,0,S02
2,46,50,29,-960,0,0,0,0,0,0,0,S02
3,62,50,29,-960,0,0,0,0,0,0,0,S02
4,78,40,29,-960,0,0,0,0,0,0,0,S02
...,...,...,...,...,...,...,...,...,...,...,...,...
15879495,1574093,50,39,-970,54,37,-989,330,-123,-922,0,S05
15879496,1574109,60,29,-960,36,55,-979,339,-123,-902,0,S05
15879497,1574125,60,39,-940,36,55,-1000,339,-123,-912,0,S05
15879498,1574140,60,39,-960,45,46,-959,320,-123,-912,0,S05


In [16]:
dataset['name'].value_counts()

S05    2527584
S01    2448984
S03    2195568
S06    2108484
S09    2067732
S07    2038320
S02    1946472
S08     546356
Name: name, dtype: int64

In [19]:
filtered_data['Action'].value_counts()

1    8275196
2    1226548
Name: Action, dtype: int64

In [17]:
filtered_data = dataset[dataset['Action'] != 0]

In [20]:
filtered_data

,time,A_F,A_V,A_L,L_F,L_V,L_L,T_F,T_V,T_L,Action,name
42879,670000,-151,990,267,63,990,80,-9,1019,0,1,S02
42880,670015,-151,1000,277,63,981,80,-9,1009,58,1,S02
42881,670031,-181,1009,247,63,981,70,-9,1019,38,1,S02
42882,670046,-161,990,237,81,981,80,0,1019,48,1,S02
42883,670062,-151,1000,257,81,981,80,-9,1000,38,1,S02
...,...,...,...,...,...,...,...,...,...,...,...,...
15872829,1469937,343,1009,118,818,481,-50,864,485,203,1,S05
15872830,1469953,313,990,128,818,527,-80,844,476,223,1,S05
15872831,1469968,333,1000,108,827,500,-20,864,466,194,1,S05
15872832,1469984,323,1039,108,836,500,-40,854,457,233,1,S05


In [28]:
X = filtered_data.iloc[:, 1:-1].values
y = filtered_data['Action'].values

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
# X_test = scaler.transform(X)

In [30]:
model = Sequential()
model.add(LSTM(units=64, input_shape=(X.shape[1], 1), return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(units=3, activation='softmax'))

In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

In [42]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [33]:
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stop])

Epoch 1/50


ValueError: in user code:

    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_f

In [56]:
model_filename = 'rnn_model.joblib'

In [44]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

25771/25771 [==============================] - 49s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test Accuracy: 1.0000


In [47]:
y_pred = model.predict(X_test)

In [ ]:
print(f'Recall for Class 1: {recall_class_1:.4f}')

In [ ]:
print(f'Recall for Class 2: {recall_class_2:.4f}')